In [ ]:
# | default_exp analysealternativemodes

In [ ]:
# | export

import sys

# from x import y syntax doesn't work because of nbdev export format
import mmon_gcm.analysing
import mmon_gcm.helper
import networkx as nx
from ipycytoscape import CytoscapeWidget

In [ ]:
def visualise_graph(graph):

    cyto = CytoscapeWidget()
    cyto.graph.add_graph_from_networkx(subgraph)

    cyto.set_style(
        [
            {
                "selector": "node",
                "style": {
                    "font-family": "helvetica",
                    "font-size": "5px",
                    "label": "data(id)",
                },
            },
            {
                "selector": "edge",
                "style": {
                    "font-family": "helvetica",
                    "font-size": "5px",
                    "label": "data(weight)",
                },
            },
        ]
    )

    # cyto.set_layout(name='circle')
    cyto.set_layout(name="cose")

    return cyto

In [ ]:
def get_reactions_correlated_to_named(reaction_to_investigate, pearson_df, threshold):

    reaction_correlations = pearson_df.loc[reaction_to_investigate]
    correlated_reactions = list(reaction_correlations.loc[abs(reaction_correlations) > threshold].index)
    correlated_df = pearson_df.loc[correlated_reactions, correlated_reactions]

    positively_correlated = correlated_df.mask(correlated_df < 0, 0)

    return positively_correlated

In [ ]:
def get_list_of_subgraphs_from_correlation_df(correlation_df, threshold):

    graph = nx.from_pandas_adjacency(mmon_gcm.analysing.get_adjacency(correlation_df, threshold))
    subgraphs_list = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]

    return subgraphs_list

As we observed variability in the flux solutions ([see solution files](outputs/model_solutions/), we wanted to see how much our conclusions rely on the assumption that the optimal solution minimises enzyme fluxes. We want to know:
- Is the variability significant? How much deviation is there?
- How typical is our pFBA solution, are our conclusions reasonable?
- What are the alternative flux modes when there is variation? Are they actually different pathways? Do they imply a different biological story?

See [Notebook 5.1](5.1_generate_alternative_flux_modes_matrix.ipynb) for the theory behind the generation of the data analysed in this notebook, based on the method for accounting for enzyme costs outlined in Cheung et al. (2015). For each scenario in the paper the model was solved for 1000 random weights, using the script in [Notebook 5.2](5.2_run_alternative_flux_modes.ipynb). This notebook analyses these solutions in two ways:
1. Looking at the pearson correlation coefficients:
   > “The flux distributions predicted by cost-weighted FBA can be used to identify reactions that operate together or in parallel by calculating the Pearson correlation coefficient (r) for the set of reaction fluxes. This approach is similar to the concepts of reaction correlation coefficient (Poolman et al., 2007) and flux correlation (Poolman et al., 2009) used in conventional FBA, and it leads to a set of r values for the small metabolic network (Table II). Reactions that operate together have a positive r (e.g. between R5 and R6); reactions that work against each other (e.g. in parallel pathways) have a negative r (e.g. between R1, R2, and R3); and reactions that support fluxes with no necessary correlation have r close to 0 (e.g. between R1 and R4). These reaction correlation coefficients complement the flux range information from FVA by characterizing sets of reactions that operate together or in parallel. While alternative metabolic routes can be easily identified by inspection in a small metabolic network (Fig. 1), this is seldom possible in large-scale models with hundreds or even thousands of reactions.” (Cheung et al., 2015, p. 2)
2. Looking at the fluxes when the solutions are averaged:
    > "However, by varying the weightings and generating an averaged solution, it is made clear that there are multiple ways in which the network might function. The averaged solution is not an optimal solution for the conventional FBA problem of flux minimization with uniform weighting (Table I shows that the sum of fluxes for a model with equal weightings is 2 units, whereas the averaged solution gives 2.161 units), but it does emphasize the potential contribution of alternative pathways. These may or may not operate simultaneously, but without further biochemical knowledge, it is not possible to rule out the possibility that all the pathways in the averaged solution contribute to the flux distribution." (Cheung et al., 2015, p. 2)

NaN is observed for correlation where there is no standard deviation in one column. 

In [ ]:
# This cell isn't exported to the .py file, so define here if running in notebook rather than as .py on e.g.a cluster

sys.argv = ["script_name",
            "blue",
            False,
            False,
            "../inputs",
            "../outputs",
            "../models"]

In [ ]:
# | export

light = sys.argv[1]
atpase_constrained = mmon_gcm.helper.convert_true_false(sys.argv[2])
starch_knockout = mmon_gcm.helper.convert_true_false(sys.argv[3])
input_dir = sys.argv[4]
output_dir = sys.argv[5]
model_dir = sys.argv[6]

map_path = input_dir + "/map.json"
json_model = model_dir + "/4_stage_GC.json"

atpase_dict = {
    True: "constrained",
    False: "unconstrained"
}

starch_dict = {
    True: "ko",
    False: "wt",
}

In [ ]:
# | export

alternative_modes_df, pfba_df, averages_df = mmon_gcm.analysing.get_solution_dfs(
    light, atpase_constrained, starch_knockout, output_dir
)

## How different is the average solution different from the pFBA solution?

- This will tell us whether our use of pFBA, and the way the solver picks equivalent reactions, has biased the reactions that show flux in the solutions we present
- As the method is designed to add variability, comparing with the average is better than comparing with the maximal/minimal solutions, because the further from the mean the more biologically-unrealistic the solution becomes. This is the range that FVA gives.
- We use percentage difference rather than absolute to see differences that would meaningfully affect our conclusions. If there are absolute values that are important then they will still show up because they will make up a big proportion of the reaction that they are important to, even if it's small for other reactions.

In [ ]:
# | export

percentage_difference_df = mmon_gcm.analysing.get_percentage_difference_df(pfba_df, averages_df)

In [ ]:
# | export

map = mmon_gcm.analysing.get_difference_map(percentage_difference_df, map_path, json_model)
map

Builder(reaction_data={'EX_X_CO2_t_gc_2': 0.0, '2KG_OAA_mc_gc_2': 100.0, 'F16ALDOLASE_RXN_c_gc_2': 0.0, 'PYRUV…

## Do the fluxes that decrease (in absolute terms) from pFBA to the average flux solution, correlate with one another?

Basically what we're looking at here is how many different "pFBA pathways" potentially have alternatives. Is it all one pathway or different ones?

In [ ]:
# | export

significance_threshold = (
    0.1  # set threshold above which a correlation is considered to exist
)

Here we only calculate the pearson for the reactions that are different, to speed things up:

In [ ]:
# | export

pearson_df = mmon_gcm.analysing.get_pearson(alternative_modes_df.loc[percentage_difference_df.index])

In [ ]:
# | export

reactions_higher_in_pfba = percentage_difference_df["% Difference"] < 0
pfba_variable_pathways = pearson_df.loc[
    reactions_higher_in_pfba, reactions_higher_in_pfba
]

This doesn't currently work because of this bug: https://github.com/zakandrewking/escher/issues/377
pfba_variable_pathways.style.apply(
    lambda x: [
        "background: red" if abs(x[v]) > significance_threshold and v != x.name else ""
        for v in x.index
    ],
    axis=1,
)

In [ ]:
# | export

G = nx.from_pandas_adjacency(
    mmon_gcm.analysing.get_adjacency(pfba_variable_pathways, significance_threshold)
)
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]

In [ ]:
subgraph = G

cyto = CytoscapeWidget()
cyto.graph.add_graph_from_networkx(subgraph)

cyto.set_style(
    [
        {
            "selector": "node",
            "style": {
                "font-family": "helvetica",
                "font-size": "5px",
                "label": "data(id)",
            },
        },
        {
            "selector": "edge",
            "style": {
                "font-family": "helvetica",
                "font-size": "5px",
                "label": "data(weight)",
            },
        },
    ]
)

# cyto.set_layout(name='circle')
cyto.set_layout(name="cose")

cyto

CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'style': {'font-famil…

In [ ]:
# | export

map = mmon_gcm.analysing.get_pfba_pathways_map(S, map_path, json_model)
map.save_html(
    output_dir +
    f"/alternative_weighting/analysis/{light}_{atpase_dict[atpase_constrained]}_{starch_dict[starch_knockout]}_pfba_pathways.html"
)
map

Builder(reaction_data={'Pi_PROTON_mc_gc_2': 0, 'H_mc_gc_2': 0, 'MAL_Pi_mc_gc_2': 0, 'MALIC_NAD_RXN_m_gc_2': 0,…

In [ ]:
import os

import pandas as pd

In [ ]:
# | export

def get_reactions_to_investigate(csv_path):

    if not os.path.exists(csv_path):
        df = pd.DataFrame(columns=["Threshold"])
        df.index.name = "Reactions"
        df.to_csv(csv_path)

    df = pd.read_csv(csv_path, index_col=0)

    return df

In [ ]:
csv_path = input_dir + \
    f'/alternative_weights_reactions/{light}_{atpase_dict[atpase_constrained]}_{starch_dict[starch_knockout]}.csv'
reactions_threshold_df = get_reactions_to_investigate(csv_path)

In [ ]:
for row in reactions_threshold_df.iterrows():
    reaction_to_investigate = row[0]
    threshold = row[1][0]

    correlation_df = get_reactions_correlated_to_named(reaction_to_investigate, pearson_df, threshold)

    subgraphs_list = get_list_of_subgraphs_from_correlation_df(correlation_df, threshold)

    alternatives_map = mmon_gcm.analysing.get_pfba_pathways_map(subgraphs_list, map_path, json_model)

    alternatives_map.save_html(
        output_dir + f'/alternative_weighting/analysis/{light}_{atpase_dict[atpase_constrained]}_{starch_dict[starch_knockout]}_{reaction_to_investigate}.html')

In [ ]:
visualise_graph(subgraphs_list)

CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'style': {'font-famil…

## Notes on alternative pathways:

### blue_unconstrained_wt

- MALIC_NAD_RXN_m_gc_2: Needed to use threshold of 0.4 to separate pathways, but can see that it's PEP carboxykinase + PEP dephosphorylase vs MALIC NAD. Although the conversion of MAL->OX in the mitochondria pops up for only the PEP carboxykinase route, this must also be used for MALIC.
- ACONITATEDEHYDR_RXN_m_gc_2: At 0.1 threshold there's a lot of mess but drop it down to 0.4 and it's just whether or not this reaction happens in the mitochondrion or cytoplasm
- SUCCCOASYN_RXN_m_gc_2: At 0.1 this includes some other reactions, but at 0.2 with pFBA this stage of the TCA cycle produces ATP which is then converted to GTP. The alternative is the direct production of GTP. Presumably this offers some sum-of-fluxes benefit because it doesn't require dealing with GTP.
- PHOSGLYPHOS_RXN_c_gc_2: Used 0.25, and looks like this is just a question of whether ATP is produced in the plastid or in the cytoplasm, and shuffling it around.
- 3PGAREARR_RXN_c_gc_2: Looks like the same as above

### blue_unconstrained_ko

This looks the same as blue_unconstrained_wt. As the solutions were the same that makes sense.

### blue_constrained_wt

- MALIC_NAD_RXN_m_gc_2: Needed to use threshold of 0.4 to separate pathways, but can see that it's PEP carboxykinase + PEP dephosphorylase vs MALIC NAD. Although the conversion of MAL->OX in the mitochondria pops up for only the PEP carboxykinase route, this must also be used for MALIC.
- ACONITATEDEHYDR_RXN_m_gc_2: At 0.1 threshold there's a lot of mess but drop it down to 0.4 and it's just whether or not this reaction happens in the mitochondrion or cytoplasm
- SUCCCOASYN_RXN_m_gc_2: At 0.1 this includes some other reactions, but at 0.2 with pFBA this stage of the TCA cycle produces ATP which is then converted to GTP. The alternative is the direct production of GTP. Presumably this offers some sum-of-fluxes benefit because it doesn't require dealing with GTP.

### blue_constrained_ko

- RXN_1461_v_gc_2 (Sucrose breakdown in vacuole): Alternative is an equiavlent pathway via fructan that has a higher sum-of-fluxes.
- K_c_gc_Linker_2: This is to do with basal osmolarity. K+ is correlated with all of the charged osmolytes in the cytoplasm, then alternatives are the others that can accumulate in the cytoplasm.
- PEPDEPHOS_RXN_c_gc_2: Needed to use threshold of 0.4 to separate pathways, but can see that it's PEP carboxykinase + PEP dephosphorylase vs MALIC NAD. Although the conversion of MAL->OX in the mitochondria pops up for only the PEP carboxykinase route, this must also be used for MALIC.
- ACONITATEDEHYDR_RXN_m_gc_2: This is just whether the reaction happens in the mitochondria or whether everything is shuttled to the cytoplasm to do it and then shuttled back again
- SUCCCOASYN_RXN_m_gc_2: With pFBA this stage of the TCA cycle produces ATP which is then converted to GTP. The alternative is the direct production of GTP. Presumably this offers some sum-of-fluxes benefit because it doesn't require dealing with GTP.

### white_constrained_wt

- PHOSGLYPHOS_RXN_c_gc_2: This is just whether it happens in the plastid or the cytosol
- TRIOSEPISOMERIZATION_RXN_p_gc_2: Again, just whether it happens in the plastid or the cytosol

## white_constrained_ko

- PHOSGLYPHOS_RXN_c_gc_2: This is just whether it happens in the plastid or the cytosol
- TRIOSEPISOMERIZATION_RXN_p_gc_2: Whether it happens in the plastid or the cytosol
- GLUC1PURIDYLTRANS_RXN_c_gc_2: Needs to be 0.52 to separate into two subgraphs. Basically two ways of getting between GLC and GLC_1_P. One via UDP and one via GDP glucose.
- K_c_gc_Linker_2:This is to do with basal osmolarity. K+ is correlated with all of the charged osmolytes in the cytoplasm, then alternatives are the others that can accumulate in the cytoplasm.
- SER_pc_gc_2: Whether SER is used or is converted to GLY and used.
- RXN_1461_v_gc_2: Whether to use an equivalent reaction via FRUCTAN.

## nops_constrained_wt

- PHOSGLYPHOS_RXN_c_gc_2: Used 0.32, whether it's in the plastid or cytosol.
- TRIOSEPISOMERIZATION_RXN_p_gc_2: Same as above, including 0.32. Just whether it's in plastid  or cytosol.
- K_v_gc_Linker_2: Used 0.11. This is just shuffling things between c and v. 
- SUCCCOASYN_RXN_m_gc_2: With pFBA this stage of the TCA cycle produces ATP which is then converted to GTP. The alternative is the direct production of GTP. Presumably this offers some sum-of-fluxes benefit because it doesn't require dealing with GTP.

## nops_constrained_ko

- K_v_gc_Linker_2: Used 0.11. This is just shuffling things between c and v.
- SUCCCOASYN_RXN_m_gc_2: With pFBA this stage of the TCA cycle produces ATP which is then converted to GTP. The alternative is the direct production of GTP. Presumably this offers some sum-of-fluxes benefit because it doesn't require dealing with GTP.